# Parse .srt files and insert in database
The subtitle files are cleaned and then the movies, subtitles, countries, genres are added to the database

In [ ]:
import pandas as pd
import pysrt
import os
import pymysql
import re

In [ ]:
FOLDER_IMG = "/home/tanguy/data/movizz/img"
FOLDER_CSV = "/home/tanguy/data/movizz/csv"
FOLDER_SRT = '/home/tanguy/data/movizz/srt'

In [ ]:
df_movies = pd.read_csv(os.path.join(FOLDER_CSV, 'metadata_movies.csv'), sep=';', index_col=0)

### Function to clean subtitles

In [ ]:
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

def striphtml2(data):
    p = re.compile(r'{.*?}')
    return p.sub('', data)

def process_quote(s):
    """
    Clean quotes
    """
    s = s.replace("<i>","").replace("</i>","").replace("<b>","").replace("</b>","")
    s = striphtml(s)
    s = striphtml2(s)
    return s

def get_filtered_subs(srt_file):
    try:
        subs = pysrt.open(srt_file)
    except UnicodeDecodeError:
        subs = pysrt.open(srt_file, encoding='iso-8859-1')
    
    # Get text
    subs_text = [process_quote(s.text) for s in subs]
    
    # To remove "Translation by.." or movie title / director
    subs_text = subs_text[3:-3]
    
    # Filter opensubtitle ads
    subs_text = [s for s in subs_text if not re.match('.*(www.OpenSubtitles.org).*', s)]
    
    # Keep the 1/3 longest quotes
    limit = int(len(subs_text)/3)
    filtered_subs = sorted(subs_text, key=len, reverse=True)[:limit]
    
    return filtered_subs

### Insert in database

In [ ]:
# Connect to the database
connection = pymysql.connect(host='localhost',user='root',password='',db='quizz_db',
                             charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)


for ttmovie_id, row in df_movies.iterrows():
    print(ttmovie_id, end = ' ')
    
    # Check if subtitles are available
    list_folder_srt = [filename for filename in [files for root, dirs, files in os.walk(FOLDER_SRT)]][0]
    
    if f'{ttmovie_id}.srt' not in list_folder_srt:
        print('[ABORT - subtitles not available]')
        continue
        
    ##########################################    
    # Format subtitles
    ##########################################
    srt_file = os.path.join(FOLDER_SRT, f'{ttmovie_id}.srt')
    subs = get_filtered_subs(srt_file)
    if len(subs) == 0:
        # TODO : handle {} format
        print('[ABORT - bad format subtitles]')
        continue
    
    ##########################################
    # Add movie in database
    ##########################################
    name = row['movie_title']
    director = row['director']
    year = int(row['title_year'])
    popularity = int(row['rating_count'])
    
    with connection.cursor() as cursor:
        cursor.execute(f"SELECT id from quizz_movie WHERE imdb_id = '{ttmovie_id}'")
        res = cursor.fetchall()
    connection.commit()
    
    if len(res) == 0:
        with connection.cursor() as cursor:
            # Create a new record
            sql = "INSERT INTO `quizz_movie` (`imdb_id`, `name`, `director`, `year`, `popularity`, `image`, `has_quote`) VALUES (%s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (ttmovie_id, name, director, year, popularity, f'covers/{ttmovie_id}.jpg', 0))
            id_movie_db = cursor.lastrowid
        connection.commit()
    else:
        id_movie_db = res[0]['id']
        
    ##########################################    
    # Add genres
    ##########################################
    list_genre = row['list_genre'].split(',')
    
    for genre in list_genre:
        # Add genre on database if doesn't exist
        with connection.cursor() as cursor:
            cursor.execute(f"SELECT id from quizz_genre WHERE name = '{genre}'")
            res = cursor.fetchall()
        connection.commit()
    
        if len(res) == 0:
            with connection.cursor() as cursor:
                sql = "INSERT INTO `quizz_genre` (`name`) VALUES (%s)"
                cursor.execute(sql, (genre))
                genre_id = cursor.lastrowid
            connection.commit()
        else:
            genre_id = res[0]['id']
            
        # Add on moviegenre
        with connection.cursor() as cursor:
            sql = "INSERT INTO `quizz_moviegenre` (`movie_id`, `genre_id`) VALUES (%s, %s)"
            cursor.execute(sql, (id_movie_db, genre_id))
        connection.commit()

    ##########################################    
    # Add countries
    ##########################################
    list_country = row['list_country'].split(',')
    for country in list_country:
        # Add genre on database if doesn't exist
        with connection.cursor() as cursor:
            cursor.execute(f"SELECT id from quizz_country WHERE name = '{country}'")
            res = cursor.fetchall()
        connection.commit()
    
        if len(res) == 0:
            with connection.cursor() as cursor:
                sql = "INSERT INTO `quizz_country` (`name`) VALUES (%s)"
                cursor.execute(sql, (country))
                country_id = cursor.lastrowid
            connection.commit()
        else:
            country_id = res[0]['id']
            
        # Add on moviegenre
        with connection.cursor() as cursor:
            sql = "INSERT INTO `quizz_moviecountry` (`movie_id`, `country_id`) VALUES (%s, %s)"
            cursor.execute(sql, (id_movie_db, country_id))
        connection.commit()
    
    
    
    ##########################################    
    # Add subtitles in database
    ##########################################

    for s in subs:
        with connection.cursor() as cursor:
            # Create a new record
            try:
                sql = "INSERT INTO `quizz_quote` (`movie_id`, `quote_text`) VALUES (%s, %s)"
                cursor.execute(sql, (id_movie_db, s))
            except:
                print('E',end='')

        connection.commit()
        
    ##########################################    
    # Add has_quote field
    ##########################################
    
    with connection.cursor() as cursor:
        cursor.execute(f"SELECT count(*) from quizz_quote WHERE movie_id = '{id_movie_db}'")
        res = cursor.fetchall()
    connection.commit()
    
    if res[0]['count(*)'] != 0:
        with connection.cursor() as cursor:
            cursor.execute(f"UPDATE quizz_movie SET has_quote=1 WHERE id='{id_movie_db}'")
        connection.commit()
        
    print('[OK]')
    
connection.close()